## DEMO

In [4]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
from utils import create_document_embeddings, get_document_embeddings

In [2]:
PATH_TO_EMBEDDINGS = r"../data/embeddings"

In [5]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [57]:
create_document_embeddings(model, [["Hello, world!", "Goodbye. This is the end."], ["This is a test", "This is not a test"]], PATH_TO_EMBEDDINGS, "demo")


array([[-2.09693518e-02,  5.88474944e-02,  2.82677300e-02,
         2.62138695e-02, -9.06362943e-03, -7.64047354e-02,
         5.53913191e-02, -3.68839912e-02, -7.84823671e-04,
        -1.27433864e-02,  1.58711728e-02,  2.14681029e-02,
         6.23353664e-03,  8.90833605e-03, -7.68442303e-02,
         8.39350000e-03, -1.81107409e-02,  1.22036822e-02,
        -1.23330548e-01,  2.18525846e-02, -2.26291306e-02,
         4.76487502e-02, -2.50382796e-02,  3.09831463e-02,
        -5.96614145e-02,  3.93032730e-02,  4.28193845e-02,
         2.59722732e-02, -1.50488382e-02, -6.17700703e-02,
         2.94973440e-02,  4.22532260e-02,  3.13867591e-02,
        -2.40303874e-02,  9.24553722e-03,  6.24796227e-02,
        -6.07810020e-02, -7.88905621e-02,  2.15569939e-02,
         7.95919355e-03,  1.69848315e-02, -1.63327418e-02,
        -4.67240810e-02, -1.32707031e-02,  2.02383064e-02,
         2.97268182e-02, -2.98732631e-02, -1.25523005e-02,
         8.27551261e-02,  5.37742898e-02, -1.58221647e-0

In [58]:
l_embs = get_document_embeddings(r"../data/embeddings/demo.npy")

## Testing REGEX

In [ ]:
import re

In [ ]:
content = 'compression quality, or color which are capable enough to dramatically modify or restructure the underlying image properties111https://www.nytimes.com/interactive/2023/06/28/technology/ai-detection-midjourney-stable-diffusion-dalle.html?auth=register-google&utm_source=pocket-newtab-intl-en. Image forgeries are almost always followed by these operations like adding some noise or applying JPEG compression to obscure any traces of image generation or forgeries and thereby'

# content = re.sub(r'\[(\d+[,]?)+\]([^\w\t])', r'\2', content) # matches when punctuation follows the citation
# content = re.sub(r'\[(\d+[,]?)+\](\w+)', r' \2', content)
content = re.sub(r'(\d+)?http[s]?:\/\/[^\s]+([^\.\?\!\,:; ])', r'', content)
# content = re.sub(r'(\d+)?(http[s]?|file):\/\/(^.\s)*([\.,]+)?', r'\3', content)
print(content)

In [ ]:
text = r'''Due
to the efficiency of hierarchical architectures, U-Net-based Diffusion Models (DMs)[1]have achieved remarkable performance in visual generation tasks[2,3,4].
Recently, inspired by the success of Transformers[5,6,7], Transformer-based DMs dubbed Diffusion Transformers (DiTs) have been developed[8]and exhibit great scalability on more complex generation tasks. Particularly, the state-of-the-art (SOTA) generation framework Sora[9]is built upon DiTs, highlighting their great potential and effectiveness.
However, their large model size and intensive computations involved in the iterative denoising process result in slow inference speeds[10], calling for effective model compression methods.'''


## Official

In [1]:
from pathlib import Path
import numpy as np
from sentence_transformers import SentenceTransformer
import logging
import re

c:\Users\rooty\UWEC\Fall 24\CS 426\GroupProject\GroupProject\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


**TODO**
- Skip poorly fomarted papers
- Split paper by SECTION rather than '\n' char
- Remove other citation types (e.g. (Name et. al., year))
- Obtain Keywords from first section if possible
- Make section embedding Hierarchical (combine the embeddings of subections!)
- Add better ending condition (The end of the paper is not always denoted by a SECTION)
- Conver ending_words matching to regex
- Adjust multi_sec_embed to create an list of ndarrays before concatenating
- Change how the `preprocess` and `is_paper_good` methods are used (e.g. change them to be class methods)

**Potential for cleaning via HTML Parsing**
- Remove Code
- See when paper content ends?
- Remove citations
- Remove links

**Problem Papers**
- 2411.13913v1_content (Unusable)
- 2403.11585v3_content (Should end earlier)


**Embeddings**
- Each embedding file contains a collection of embeddings with shape = (num_of_secs, emb_size). NOTE THIS IS SUBJECT TO CHANGE!
- The last embedding in the file is the papers mean embedding.


**Model Info**
- https://huggingface.co/sentence-transformers/sentence-t5-large

**Things to Read**
- https://huggingface.co/docs/transformers/model_memory_anatomy
- https://huggingface.co/docs/transformers/perf_train_gpu_one
- https://huggingface.co/docs/transformers/llm_tutorial_optimization

**Training End Dates**
- LLAMA : Dec. 2023
- sentence-t5-large : ??


In [83]:
path_to_papers = Path(r'../papers')
path_to_embs = Path(r'../data/embeddings')

END_OF_PAPER_WORDS = [
    'references',
    'acknowledgements', 
    'acknowledgement',
]

papers_to_skip = [
    '2311.11329v2_content.txt',
    '2411.09324v2_content.txt',
    '2411.14259v1_content.txt'
]


In [84]:
model = SentenceTransformer('sentence-transformers/sentence-t5-large', device='cuda')

In [76]:
emb_lib = EmbeddingLibrary(
    path_to_papers=path_to_papers,
    path_to_embs=path_to_embs,
    model=model,
    end_of_paper_words=END_OF_PAPER_WORDS,
    papers_to_skip=papers_to_skip,
    name="Prototype"
)

emb_lib.update_full_paper_embs()

In [80]:
x = model.encode(['RF jamming transmits radio signals indiscriminately across a range of frequencies, causing interference and disrupting communication. Jamming can be broadly categorized as active jamming and reactive jamming. Active jamming continuously emits powerful interference signals, but its continuous operation leaves detectable traces, making it vulnerable to defensive techniques. Reactive jamming adjusts its jamming behavior according to observed signals in the environment. It remains silent when the channel is idle but initiates high-power signal transmission upon detecting activity on the channel. The drawback of these approaches is that spectrum owners may promptly detect the presence of an attack and respond accordingly.'])

y = model.encode(['The cat was fat. So was the dog.', 'He ran so fast. But not fast enough. Sad fat dog'])

In [82]:
y.shape

(2, 768)

In [77]:
query = ' the research provides an innovative and efficient solution for code generation from ML task descriptions, showcasing the capabilities of Linguacodus. By capitalizing on the Code4ML dataset’s wealth of resources and introducing a structured approach to instruction synthesis and code generation, we bridge the gap between natural language task descriptions and executable code, making machine learning development more accessible and efficient'

emb_lib.search_papers(query)

tensor([[0.7425, 0.7420, 0.7429, 0.7443, 0.7437, 0.7426, 0.7426, 0.7418, 0.7421,
         0.7428, 0.7431, 0.7434, 0.7432, 0.7447, 0.7426, 0.7425, 0.7430, 0.7425,
         0.7437, 0.7419, 0.7440, 0.7423, 0.7412, 0.7420, 0.7433, 0.7431, 0.7438,
         0.7427, 0.7441, 0.7427, 0.7429, 0.7430, 0.7430, 0.7424, 0.7423, 0.7447,
         0.7436, 0.7422, 0.7425, 0.7439, 0.7442, 0.7432, 0.7438]])


['2411.09598v1',
 '2403.12778v2',
 '2411.09101v1',
 '2411.09702v1',
 '2309.01837v3']

In [ ]:
from pathlib import Path
import numpy as np
from sentence_transformers import SentenceTransformer, SimilarityFunction
from embeddings.embedding_library import EmbeddingLibrary
import logging
import re

In [ ]:
path_to_papers = Path(r'papers')
path_to_embs = Path(r'data/embeddings')

# should all be lower case
end_of_paper_words = [
    'references',
    'acknowledgements', 
    'acknowledgement',
]

papers_to_skip = [
    '2311.11329v2_content.txt',
    '2411.09324v2_content.txt',
    '2411.14259v1_content.txt'
]

In [ ]:
model = SentenceTransformer('sentence-transformers/allenai-specter', device='cuda')

In [ ]:
emb_lib = EmbeddingLibrary(
    path_to_papers=path_to_papers,
    path_to_embs=path_to_embs,
    model=model,
    end_of_paper_words=end_of_paper_words,
    papers_to_skip=papers_to_skip,
    name="Prototype",
    path_to_log=Path("./log")
)

In [ ]:
emb_lib.multi_sec_embed(skip_existing=False)

Batches:  50%|█████     | 674/1335 [00:05<00:05, 117.07it/s]


KeyboardInterrupt: 

In [ ]:
emb_lib.update_full_paper_embs()

In [ ]:
model.similarity_fn_name = SimilarityFunction.COSINE

In [ ]:
x = emb_lib.full_paper_embs

model.similarity(x, x)

tensor([[1.0000, 0.9992, 0.9996,  ..., 0.9993, 0.9994, 0.9991],
        [0.9992, 1.0000, 0.9995,  ..., 0.9993, 0.9994, 0.9994],
        [0.9996, 0.9995, 1.0000,  ..., 0.9994, 0.9995, 0.9991],
        ...,
        [0.9993, 0.9993, 0.9994,  ..., 1.0000, 0.9994, 0.9994],
        [0.9994, 0.9994, 0.9995,  ..., 0.9994, 1.0000, 0.9989],
        [0.9991, 0.9994, 0.9991,  ..., 0.9994, 0.9989, 1.0000]])

In [ ]:
import umap
import umap.plot


mapper = umap.UMAP().fit()

p = umap.plot.points(mapper, labels=data.Cat1)
umap.plot.show(p)

In [ ]:
emb_lib.search_papers(
    '',
    'Inspired by the capacity of self-attention to capture patch-level interactions, we propose a 4D interaction encoder for efficient human-scene interaction estimation. In Fig.3, we visualize the attention map of the token overlapped with the head of a person and the feature map output by the last block of a DINOv2dinov2pre-trained ViT.Rich semantic information in the feature map enables effective distinction between objects. However, it cannot represent interactions between image regions. In contrast, a specific token’s attention map is capable of acquiring interaction information between it and other regions. Therefore, we propose the interaction encoder to leverage attention maps, which are referred to as 4D interaction features, from pre-trained ViTs with a simple adaptation (Fig.4(a)). Compared with the encoders used in the previous tasks (Fig.4(b)) such as object detectionvitdetand image mattingvitmattethat utilize feature maps to distinguish objects, the interaction feature-based encoder inherently facilitates the explicit capture of patch relations, which is required by gaze following. Given an image with a resolution of. The interaction encoder extracts 4D features, which explicitly describe the interactions between image patches. The correlations among them are represented as a 4D tensor with a size of, whereand, anddenotes the patch-size of the ViT. Different from the final feature map leveraged in other regionsvitdet;vitpose;vitmatte, this 4D representation reflects the inner-token relations which are more effective in gaze following.'
)

['2410.23279v3',
 '2411.14354v1',
 '2410.14845v2',
 '2312.09440v2',
 '2308.07279v2']

In [ ]:
emb_lib.paper_ids

46